Execute code below once to get latest `Debug Bridge` driver

In [ ]:
!wget https://raw.githubusercontent.com/Xilinx/PYNQ/master/pynq/lib/debugbridge.py -O debugbridge.py
!mv  debugbridge.py /usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/lib/debugbridge.py

Download overlay and grab DMA aliases

In [ ]:
from pynq import Overlay, allocate, PL
import numpy as np

PL.reset()

ol = Overlay('dma_dbridge.bit')
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel

Show all available IP

In [ ]:
ol.ip_dict

Bind `Debug Bridge` IP to the PYNQ driver

In [ ]:
bridge = ol.debug_bridge_0

Start `xvc_server` accepting connections from any IP

In [ ]:
bridge.start_xvc_server(bufferLen=4096, serverAddress="0.0.0.0",
                 serverPort=2542, reconnect=True, verbose=True)

Show board IP, assuming Ethernet is connected

In [ ]:
!ifconfig eth0 | grep 'inet ' | awk '{print $2}'

You now can connect to the XVC server from your Vivado instance.

Allocate buffers

In [ ]:
data_size = 16

input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)
input_buffer[:] = np.arange(data_size, dtype=np.uint32)

In Vivado Hardware Manager, set the trigger to start capturing when TVALID of *axi_dma_M_AXIS_MM2S* is 1 or in the rising edge.

In [ ]:
dma_recv.transfer(output_buffer)
dma_send.transfer(input_buffer)
dma_send.wait()

dma_recv.wait()
print(f'Are buffers equal after DMA? {np.array_equal(output_buffer, input_buffer)}')

Stop the XVC server

In [ ]:
bridge.stop_xvc_server()

Release the buffers and overlay

In [ ]:
del input_buffer, output_buffer
ol.free()